In [1]:
from ultralytics import YOLO
import torch.nn as nn
import copy
import torch
from ultralytics.nn.modules import Concat, C2f, Conv


pretrained_model = YOLO('yolov8m.pt').model
backbone = nn.Sequential(*list(pretrained_model.model.children())[:10])


In [2]:
class CustomBackbone(nn.Module):
    def __init__(self, layers, out_idx=[2, 4, 9]):
        super().__init__()
        self.layers = nn.ModuleList(layers)
        self.out_idx = out_idx
        
    def forward(self, x):
        outputs = []
        for idx, layer in enumerate(self.layers):
            x = layer(x)
            if idx in self.out_idx:
                outputs.append(x)
        return outputs

In [ ]:
backbone_rgb = CustomBackbone(backbone)
backbone_ir = copy.deepcopy(backbone_rgb)

backbone_ir.layers[0].conv = nn.Conv2d(1, 48, kernel_size=3, stride=2, padding=1, bias=False)
print(backbone_rgb.layers[0].conv.weight.shape)
print(backbone_ir.layers[0].conv.weight.shape)

torch.Size([48, 3, 3, 3])
torch.Size([48, 1, 3, 3])


In [4]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class CustomNeck(nn.Module):
    def __init__(self):
        super().__init__()
        # Przykładowe bloki do przetwarzania map cech różnych skal:
        # Przyjmujemy, że features[0] ma kształt [B, 96, 160, 160],
        # features[1] ma kształt [B, 192, 80, 80] i
        # features[2] ma kształt [B, 576, 20, 20].
        
        # Bloki dla najwyższej rozdzielczości
        self.conv_P0 = nn.Sequential(
            nn.Conv2d(96, 128, kernel_size=3, padding=1, bias=False),
            nn.BatchNorm2d(128),
            nn.ReLU(inplace=True)
        )
        
        # Bloki dla środkowej rozdzielczości
        self.conv_P1 = nn.Sequential(
            nn.Conv2d(192, 256, kernel_size=3, padding=1, bias=False),
            nn.BatchNorm2d(256),
            nn.ReLU(inplace=True)
        )
        
        # Bloki dla najniższej rozdzielczości
        self.conv_P2 = nn.Sequential(
            nn.Conv2d(576, 512, kernel_size=3, padding=1, bias=False),
            nn.BatchNorm2d(512),
            nn.ReLU(inplace=True)
        )
        
        # Następnie możesz scalić mapy cech – tutaj przykład scalenia po upsampling'u:
        self.fuse = nn.Sequential(
            nn.Conv2d(128 + 256 + 512, 512, kernel_size=1, bias=False),
            nn.BatchNorm2d(512),
            nn.ReLU(inplace=True)
        )
        
        # Przykładowa głowica detekcji – liczba wyjść zależy od Twojej implementacji
        self.detect = nn.Conv2d(512, 80, kernel_size=1)  # np. 80 może oznaczać np. klasy lub inne wyjście
    
    def forward(self, features):
        # Załóżmy, że features to lista trzech map cech z backbone: [P0, P1, P2]
        # P0: [B, 96, 160, 160]
        # P1: [B, 192, 80, 80]
        # P2: [B, 576, 20, 20]
        
        # Najpierw przetwarzamy każdą mapę cech osobno
        p0 = self.conv_P0(features[0])
        p1 = self.conv_P1(features[1])
        p2 = self.conv_P2(features[2])
        
        # Następnie dopasowujemy rozmiary przestrzenne:
        # Zakładamy, że chcemy scalić wszystkie mapy do rozmiaru najdrobniejszej mapy, ale
        # często lepszym rozwiązaniem jest dopasowanie do najwyższej rozdzielczości,
        # wtedy wykorzystując upsampling.
        # Przykład: upsamplujemy p2 do rozmiaru p0:
        p2_up = F.interpolate(p2, size=p0.shape[-2:], mode='nearest')
        p1_up = F.interpolate(p1, size=p0.shape[-2:], mode='nearest')
        
        # Teraz łączymy je wzdłuż kanału
        fused = torch.cat([p0, p1_up, p2_up], dim=1)  # wynik: [B, 128+256+512, 160, 160]
        fused = self.fuse(fused)
        
        # Ostatecznie przekazujemy scalone cechy do głowicy detekcji
        out = self.detect(fused)
        return out


In [ ]:
class CustomYOLO(nn.Module):
    def __init__(self, pretrained_model, backbone_rgb, backbone_ir):
        super().__init__()
        self.backbone_rgb = backbone_rgb
        self.backbone_ir  = backbone_ir
        
        self.neck_head = CustomNeck()
        
    def forward(self, x_rgb, x_ir):
        features_rgb = self.backbone_rgb(x_rgb)
        features_ir  = self.backbone_ir(x_ir)
        
        print(features_ir)
        print(features_rgb)
        fused_features = [f_rgb + f_ir for f_rgb, f_ir in zip(features_rgb, features_ir)]
        
        # Teraz przekazujemy listę scalonych map do neck/head
        # Twój moduł CustomNeck wie już, które mapy co oznaczają
        return self.neck_head(fused_features)

In [17]:
# Inicjalizacja modelu
model = CustomYOLO(pretrained_model, backbone_rgb, backbone_ir)

x_rgb = torch.randn(1, 3, 640, 640)
x_ir = torch.randn(1, 1, 640, 640)

# Forward pass
output = model(x_rgb, x_ir)

RuntimeError: Input type (torch.FloatTensor) and weight type (torch.cuda.FloatTensor) should be the same or input should be a MKLDNN tensor and weight is a dense tensor

In [7]:
output

tensor([[[[-2.0735, -1.8894, -2.3014,  ..., -1.2905, -1.1472, -1.2157],
          [-2.0777, -2.0100, -2.1465,  ..., -1.4722, -1.3617, -1.6204],
          [-1.3772, -1.3080, -1.4403,  ..., -1.2648, -0.8383, -1.0370],
          ...,
          [-1.3646, -0.8256, -0.8414,  ..., -1.0123, -1.5207, -1.4113],
          [-0.8432, -0.6363, -0.4322,  ..., -1.3918, -0.9484, -1.0619],
          [-0.9874, -0.9476, -0.7517,  ..., -1.7655, -1.0977, -0.6837]],

         [[ 0.7675,  1.1218, -0.2295,  ..., -0.0980,  0.4910,  0.9436],
          [ 0.9526,  0.9408,  0.2694,  ...,  0.2726,  0.5831,  0.6067],
          [ 0.6981,  0.8793,  0.1942,  ...,  0.0569,  0.4984,  0.9570],
          ...,
          [ 0.3456,  0.5022,  0.0493,  ...,  0.4837,  1.0210,  1.3509],
          [ 0.2469,  0.2135, -0.7111,  ...,  0.5428,  1.3501,  1.4189],
          [ 0.0785,  0.4662, -0.3617,  ...,  0.6158,  1.2786,  1.7035]],

         [[ 0.6137,  0.5617,  0.5906,  ...,  0.8256,  0.5149,  0.3238],
          [ 0.6676,  0.3449,  

In [8]:
import os
import cv2
import torch
from torch.utils.data import Dataset
import numpy as np
import xml.etree.ElementTree as ET

class LLVIPDataset(Dataset):
    def __init__(self, root_dir, split='train', transform=None, img_size=(640,640)):
        """
        Args:
            root_dir (str): Ścieżka do katalogu LLVIP.
            split (str): 'train' lub 'test'
            transform: opcjonalne transformacje (np. albumentations) stosowane na obrazach.
            img_size: rozmiar docelowy obrazów (może być potrzebny przy skalowaniu bounding boxów).
        """
        self.root_dir = root_dir
        self.split = split
        self.transform = transform
        self.img_size = img_size

        # Katalogi z obrazami RGB i IR
        self.rgb_dir = os.path.join(root_dir, 'visible', split)
        self.ir_dir  = os.path.join(root_dir, 'infrared', split)
        # Katalog z adnotacjami
        self.ann_dir = os.path.join(root_dir, 'Annotations')
        
        # Pobieramy listę plików; załóżmy, że rozszerzenie to .jpg
        self.filenames = [f for f in os.listdir(self.rgb_dir) if f.lower().endswith('.jpg')]
        self.filenames.sort()  # Dla spójności

    def __len__(self):
        return len(self.filenames)

    def parse_voc_annotation(self, xml_file):
        """
        Parsowanie adnotacji w formacie VOC z podanego pliku XML.
        Zwraca słownik z:
          'boxes': tensor o wymiarach [N, 4] (x_min, y_min, x_max, y_max),
          'labels': tensor o wymiarach [N], (przyjmujemy, że etykiety są konwertowane do int)
        
        Jeśli masz mapowanie nazw klas na liczby, można je dodać.
        """
        tree = ET.parse(xml_file)
        root = tree.getroot()
        
        boxes = []
        labels = []
        
        for obj in root.findall('object'):
            # Pobierz etykietę jako string
            label = obj.find('name').text
            # Przykład: przekonwertuj etykietę do liczby – tutaj zakładamy prostą konwersję lub słownik mapowania
            # Możesz stworzyć własny słownik, np. {'person': 1, 'car': 2, ...}. Dla przykładu użyjemy hash,
            # ale w praktyce lepiej jawnie zdefiniować mapowanie.
            label_int = abs(hash(label)) % 1000  # Przykładowa konwersja
            
            bbox = obj.find('bndbox')
            x_min = float(bbox.find('xmin').text)
            y_min = float(bbox.find('ymin').text)
            x_max = float(bbox.find('xmax').text)
            y_max = float(bbox.find('ymax').text)
            
            boxes.append([x_min, y_min, x_max, y_max])
            labels.append(label_int)
            
        if boxes:
            boxes = torch.tensor(boxes, dtype=torch.float32)
            labels = torch.tensor(labels, dtype=torch.int64)
        else:
            boxes = torch.zeros((0,4), dtype=torch.float32)
            labels = torch.zeros((0,), dtype=torch.int64)
            
        return {'boxes': boxes, 'labels': labels}
    
    def __getitem__(self, idx):
        filename = self.filenames[idx]
        
        # Ścieżki do obrazów oraz adnotacji
        rgb_path = os.path.join(self.rgb_dir, filename)
        ir_path  = os.path.join(self.ir_dir, filename)
        # Zakładamy, że plik XML ma taką samą nazwę, ale z rozszerzeniem .xml
        xml_filename = os.path.splitext(filename)[0] + '.xml'
        ann_path = os.path.join(self.ann_dir, xml_filename)
        
        # Wczytanie obrazów
        rgb_img = cv2.imread(rgb_path)
        rgb_img = cv2.cvtColor(rgb_img, cv2.COLOR_BGR2RGB)
        ir_img = cv2.imread(ir_path, cv2.IMREAD_GRAYSCALE)
        ir_img = np.expand_dims(ir_img, axis=-1)  # [H, W, 1]

        # Opcjonalna zmiana rozmiaru
        rgb_img = cv2.resize(rgb_img, self.img_size)
        ir_img = cv2.resize(ir_img, self.img_size)
        
        # Normalizacja
        rgb_img = rgb_img.astype(np.float32) / 255.0
        ir_img = ir_img.astype(np.float32) / 255.0
        
        # Konwersja na tensory: zmiana wymiarów: H, W, C -> C, H, W
        rgb_img = torch.from_numpy(rgb_img).permute(2, 0, 1)
        ir_img = torch.from_numpy(ir_img).permute(2, 0, 1)
        
        # Wczytanie adnotacji
        target = self.parse_voc_annotation(ann_path)
        
        # Jeżeli używasz dodatkowych transformacji (np. augumentacji) to możesz je zastosować tu
        if self.transform is not None:
            # transform powinien obsługiwać dwa obrazy. Możesz zdefiniować własną funkcję, która będzie to obsługiwać.
            augmented = self.transform(image=rgb_img.numpy(), mask=ir_img.numpy())
            rgb_img = torch.from_numpy(augmented['image']).permute(2, 0, 1)
            ir_img = torch.from_numpy(augmented['mask']).permute(2, 0, 1)
        
        return rgb_img, ir_img, target


In [9]:
# Funkcja collate_fn do DataLoadera (gdy targets mają zmienną liczbę elementów)
def collate_fn(batch):
    rgb_imgs, ir_imgs, targets = zip(*batch)
    rgb_imgs = torch.stack(rgb_imgs, dim=0)
    ir_imgs  = torch.stack(ir_imgs, dim=0)
    return rgb_imgs, ir_imgs, list(targets)

In [10]:
def dummy_loss(outputs, targets):
    """
    Funkcja straty tylko do celów demonstracyjnych.
    Zakładamy, że outputs to tensor wyjściowy modelu,
    a targets to lista adnotacji – tutaj generujemy cel jako tensor zer.
    """
    # Przykladowo celowy tensor o takim samym kształcie jak outputs
    target_tensor = torch.zeros_like(outputs)
    loss = F.mse_loss(outputs, target_tensor)
    return loss

In [11]:
def train_model(model, train_loader, optimizer, num_epochs=10, device=torch.device("cuda" if torch.cuda.is_available() else "cpu")):
    model.to(device)
    model.train()
    for epoch in range(num_epochs):
        epoch_loss = 0.0
        for batch_idx, (x_rgb, x_ir, targets) in enumerate(train_loader):
            x_rgb = x_rgb.to(device)
            x_ir  = x_ir.to(device)
            # Przenoszenie adnotacji (targets) na urządzenie
            targets = [{k: v.to(device) for k, v in t.items()} for t in targets]
            optimizer.zero_grad()
            outputs = model(x_rgb, x_ir)
            loss = dummy_loss(outputs, targets)
            loss.backward()
            optimizer.step()
            epoch_loss += loss.item()
        print(f"Epoch {epoch+1}/{num_epochs}, Loss: {epoch_loss/len(train_loader):.4f}")

In [12]:
from torch.utils.data import Dataset, DataLoader


In [13]:
# Ścieżka do katalogu LLVIP – zmodyfikuj na właściwą ścieżkę
root_dir = 'LLVIP'

# Utworzenie datasetu i DataLoadera (dla treningu)
dataset = LLVIPDataset(root_dir, split='train', img_size=(640, 640))
train_loader = DataLoader(dataset, batch_size=4, shuffle=True, collate_fn=collate_fn)

# Wczytanie pretrenowanego modelu YOLOv8m (upewnij się, że model 'yolov8m.pt' jest dostępny)
pretrained_model = YOLO('yolov8m.pt')

# Utworzenie customowego modelu
model = CustomYOLO(pretrained_model, backbone_rgb, backbone_ir)

# Konfiguracja optymalizatora
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)

# Uruchomienie treningu
num_epochs = 10
train_model(model, train_loader, optimizer, num_epochs=num_epochs)

RuntimeError: permute(sparse_coo): number of dimensions in the tensor input does not match the length of the desired ordering of dimensions i.e. input.dim() = 2 is not equal to len(dims) = 3